In [1]:
from __future__ import print_function
import torch


ModuleNotFoundError: No module named 'torch'

In [2]:
x = torch.ones(5, 3)
#print(x)
y = torch.rand(5, 3)
#print(y)
z = x+y
#print(z)
v = torch.zeros(5,3)
p = torch.add(z,v)# or torch.add(z,v, out=p)
q = torch.randn_like(p)
p.add_(q)
print(p)

NameError: name 'torch' is not defined

In [12]:
##tensor manipulation on size and slice
x = torch.zeros(5,5)
x[:, 1:3] = 1
x[1:3, :] = 8
#print(x)

y = torch.zeros(4,10)
y[1:3, 1:6] = 1
#print(y)
y = y.view(20, 2)
#print(y)
y = y.view(5, -1)
print(y.size()) #computes automatically the size when -1 spec.
z = y.view(2,4,-1)
print(z.size())

torch.Size([5, 8])
torch.Size([2, 4, 5])


In [13]:
##CUDA Tensors
x = torch.randn(5,5)
y = torch.randn(5,5)
print(torch.cuda.is_available())
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    # move a tensor on GPU
    x = x.to(device)
    y = y.to(device) # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double)) 

True


RuntimeError: cuda runtime error (48) : no kernel image is available for execution on the device at c:\programdata\miniconda3\conda-bld\pytorch_1524546371102\work\aten\src\thc\generic/THCTensorMathPointwise.cu:265

In [63]:
#autograd
x = torch.ones(2,2, requires_grad=True)
g = torch.eye(2,2, requires_grad=True)
#can also be set to True with  
#x.requires_grad_()
y = torch.ones(2,2, requires_grad=True)
#print(x.grad_fn)

y = x+g

z = y*y*3
print(z.grad_fn)

out = z.mean()

out.backward()
print(x.grad)
print(g.grad)
print(y.grad)

tensor([[ 3.0000,  1.5000],
        [ 1.5000,  3.0000]])
tensor([[ 3.0000,  1.5000],
        [ 1.5000,  3.0000]])
None


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [9]:
##Try to infer with a dummy input
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)
net.zero_grad()
out.backward(torch.randn(1, 10))

tensor([[-0.0867, -0.0521, -0.0002, -0.1221,  0.1076, -0.1152,  0.0285,
         -0.1500, -0.0436,  0.0118]])


In [23]:
##Compute the loss function
output = net(input)
target = torch.arange(1, 11)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(38.9291)


In [24]:
##Backpropagate
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad) #the grad of the first conv funct's bias 

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([ 0.,  0.,  0.,  0.,  0.,  0.])
conv1.bias.grad after backward
tensor(1.00000e-02 *
       [ 2.9273,  5.7772,  6.2429,  5.4668, -3.6424, -1.9267])


In [25]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update